In [193]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [201]:
names = ['Дата публикации','Ссылка' ,"Цена" ,"Адрес" ,"Год постройки" ,"Материал постройки" ,"Отопление",'Кол-во комнат' ,"Общая площадь" ,"Жилая площадь" ,"Площадь кухни" ,"Расположение квартиры" ,"Этаж" ,"Окна выходят" ,"Ремонт" ,"Раздельных санузлов"]

df = pd.DataFrame(columns=names)

In [210]:
page = requests.get('https://vladis.ru/syktyvkar/kupit-kvartiru?offset=')
page
src = page.text
soup = BeautifulSoup(src, 'lxml')
div = soup.find('div', 'sc-fa1b8d85-2 dKriSD')
div = div.find_all('div','sc-e0405ac0-0 cOTnmg')

Алгоритм работы функции searcher()

Функция searcher() предназначена для извлечения и структурирования данных о квартирах с веб-страниц. Её работа состоит из следующих этапов:

1.  Предварительная обработка HTML: Исходный HTML-код форматируется для обеспечения корректного парсинга.

2.  Поиск ключевых контейнеров: В структуре документа локализуются два элемента "select", которые содержат основную информацию о характеристиках квартиры.

3.  Инициализация структуры данных: Создается словарь, который будет использоваться для временного хранения и категоризации извлеченных данных.

4.  Извлечение данных: Происходит последовательный парсинг и сбор требуемых значений из найденных элементов.

5.  Формирование результата: Подготовленные данные из словаря переносятся в массив, готовый для последующей передачи и интеграции в таблицу.










In [202]:
def searcher(id, c, linkToHome):
  count = c
  iner_page = requests.get('https://vladis.ru/syktyvkar/kupit-kvartiru/'+str(id))
  print(iner_page,' == ', count)
  srci = iner_page.text
  vnr = BeautifulSoup(srci, 'lxml')

  cah = vnr.find(id='offer-card_about-house')
  caa = vnr.find(id='offer-card_about_apartment')

  slovar = {'Дата публикации': None,
            'Ссылка': None,
            'Цена': None,
            'Адрес': None,
            'Год постройки': None,
            'Материал постройки': None,
            'Отопление': None,
            'Кол-во комнат': None,
            'Общая площадь': None,
            'Жилая площадь': None,
            'Площадь кухни': None,
            'Расположение квартиры': None,
            'Этаж': None,
            'Окна выходят': None,
            'Ремонт': None,
            'Раздельных санузлов': None,
  }

  okvart = caa.find('div', 'sc-ef189e20-2 glcvYz') #данные о квартире
  for tab in okvart:
    if tab.find('p','sc-ec3d9fa2-0 jPzElo').text in slovar:
      slovar[tab.find('p','sc-ec3d9fa2-0 jPzElo').text] = tab.find('span', 'sc-7cc6aa8f-0 duICqo').text

  ohous = cah.find('div', 'sc-ef189e20-2 glcvYz') #данные о доме
  for tab in ohous:
    if tab.find('p','sc-ec3d9fa2-0 jPzElo').text in slovar:
      slovar[tab.find('p','sc-ec3d9fa2-0 jPzElo').text] = tab.find('span', 'sc-7cc6aa8f-0 duICqo').text

  slovar['Дата публикации'] = vnr.find('div', 'sc-f7f5b7ea-2 kuneej').text
  slovar['Цена'] = vnr.find('div', 'sc-e25f85f-1 frDpxM price-title').text.replace('\xa0', '')
  slovar['Адрес'] = vnr.find('div', 'sc-ac6b7eb0-1 hYEXQv').text.replace('\xa0', '')
  slovar['Кол-во комнат'] = ((vnr.find('h1', 'sc-ec3d9fa2-0 lcymZ').text).split())[1].replace(',','')
  slovar['Ссылка'] = linkToHome
  data = []
  for x in slovar:
    data.append(slovar.get(x))

  df.loc[count] = data
  count += 1


Перед непосредственным извлечением данных осуществляется подготовительный этап:
Инициализация функции **searcher()**: В качестве аргументов функции передаются:

*   Идентификатор страницы (ID), однозначно определяющий обрабатываемый документ.
*   Длина целевой таблицы, необходимая для корректной индексации новых записей.
*   Прямая ссылка (URL) на обрабатываемую веб-страницу.




In [203]:
def finder():
  for link in div:
    id = link.find('a', 'sc-e0405ac0-19 cMEHDC').get('href')[-7:]
    LTH = link.find('a', 'sc-e0405ac0-19 cMEHDC').get('href')
    searcher(id, len(df), LTH)


Для полного охвата данных осуществляется циклический обход всех целевых страниц с помощью функции **finder()**, которая последовательно применяет алгоритм парсинга к каждому элементу.


In [ ]:
for i in range(0, 11):
  print(requests.get(
        'https://vladis.ru/syktyvkar/kupit-kvartiru?offset='+str(i*40)
  ))
  finder()

Конечный результат превращаем в csv файл

In [ ]:
df

In [209]:
df.to_csv('house.csv', index=False)